<a href="https://colab.research.google.com/github/GMS9github/GMS/blob/main/GMS_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the Function to scrape the Url Page
def scrape_page(url, Blog_Titles, Blog_Dates, Blog_Image_Urls, Blog_Likes_Counts):
    response = requests.get(url, headers=headers)#Get Response from url
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')# Parse the html content using BeautifulSoup Library
        blog_posts = soup.find_all('article', class_=lambda x: x and x.startswith('blog-item category-blog')) #Find all elements of type article that starts with 'blog-item category-blog'
        # Iterate over blog_posts using for loop
        for post in blog_posts:
            # Extract title
            title = post.find('h6') # Find header which holds the blog title
            Blog_Titles.append(title.text.strip() if title else 'N/A')#Append the header to list called in function scrape_page

            # Extract date
            Blog_detail = post.find('div', class_='bd-item') # Find the class which holds the blog date(here it is 'bd-item')
            # The above Class contains different icons , so comparing with the icon that holds the Blog Date(here it is 'material-design-icon-history-clock-button')
            if Blog_detail.find('i',class_='material-design-icon-history-clock-button'):
              date = Blog_detail.find('span')
              Blog_Dates.append(date.text.strip() if date else 'N/A')#append the date to list called in function scrape_page

            # Extract image URL
            img_div = post.find('div', class_='img')# Find the class which holds the blog date(here it is 'img')
            #The above image class has url in 'data-bg' element
            img_url = img_div.find('a')['data-bg'] if img_div else 'N/A'
            Blog_Image_Urls.append(img_url if img_url else 'N/A')#append the date to list called in function scrape_pag

            # Extract likes count
            Blog_Likes_detail = post.find('a' , class_='zilla-likes')# Find the class which holds the blog date(here it is 'zilla-like')
            # The above Class contains different icons , so comparing with the icon that holds the Blog Date(here it is 'material-design-icon-favorite-heart-outline-button')
            if Blog_Likes_detail.find('i',class_='material-design-icon-favorite-heart-outline-button'):
              likes = Blog_Likes_detail.find('span')
              Blog_Likes_Counts.append(likes.text.strip() if likes else 'N/A')#append the date to list called in function scrape_pag

        #Get the url to the Next Page Which is present in Pagination Class
        next_page_link = soup.find('a', class_='next page-numbers') # Next page-numbers sub class of Pagination class has the url to the next page
        if next_page_link:
            next_page_url = next_page_link.get('href')#Get the URL from the above link
            scrape_page(next_page_url, Blog_Titles, Blog_Dates, Blog_Image_Urls, Blog_Likes_Counts)#Recursively call until the last page
    else:
        print(f'Failed to retrieve the webpage: {url}')

# Declare URL and header
url = 'https://rategain.com/blog'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}   # The 'User-Agent' header is used to mimic a real browser (like Chrome) when making requests to the website

# Declare lists to save the scraped members
Blog_Titles = []
Blog_Dates = []
Blog_Image_Urls = []
Blog_Likes_Counts = []

scrape_page(url, Blog_Titles, Blog_Dates, Blog_Image_Urls, Blog_Likes_Counts)

# Create a Dictionary from the above lists
Excel_Data = {
    'Blog Title': Blog_Titles,
    'Blog Date': Blog_Dates,
    'Blog Image URL': Blog_Image_Urls,
    'Likes Count': Blog_Likes_Counts
}

# Create a DataFrame from the extracted data
df = pd.DataFrame(Excel_Data)
# Save the data to an Excel file
df.to_excel('Blog_Data.xlsx', index=False)
print('Data saved to Blog_Data.xlsx')

Data saved to Blog_Data.xlsx
